In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install implicit

In [0]:
import pandas as pd
CheckinInfo=pd.read_csv('/content/gdrive/My Drive/Dataset/checkin_bj_uniq.txt',header=None,sep='\t')
CheckinInfo.columns=['User_id','Time','latitude','longititude','Item_id']
CheckinInfo = CheckinInfo.loc[:3093551]
#1000 - 74960
#5000 - 337240
#10000 - 655759
#20000 - 1263635
#30000 - 1827668
#40000 - 2432870
#50000 - 3093551

In [0]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

a1=CheckinInfo.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()

person_c = CategoricalDtype(sorted(a1.User_id.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(a1.Item_id.unique()), ordered=True)

user_lbl=list(sorted(a1.User_id.unique()))
item_lbl=list(sorted(a1.Item_id.unique()))


#Converting sparse-matrix to dataframe 
#dfs = pd.SparseDataFrame(sparse_checkin, index=person_c.categories, columns=thing_c.categories,default_fill_value=0)

In [0]:
User_place_count=a1.groupby( [ "User_id"] ).size().to_frame(name = 'count').reset_index()
User_place_count.count = User_place_count["count"].astype('int64')
User_place_count.User_id = User_place_count["User_id"].astype('int64')


#generating single user dictionary
dict_singleUser=[]
for index,row in User_place_count[:][User_place_count["count"] == 1].iterrows():
    dict_singleUser.append(row.User_id)

In [0]:
dict_singleUser2=[]
for i in dict_singleUser:
    if CheckinInfo[:][CheckinInfo.User_id == i].User_id.count() == 1:
      dict_singleUser2.append(i)

In [0]:
# hitrate - one visit hide
from sklearn.model_selection import train_test_split

temp = CheckinInfo[~CheckinInfo.User_id.isin(dict_singleUser2)]
train, test = train_test_split(temp, test_size=0.1)
train = CheckinInfo.drop(test.index.values.tolist())
a1_train=train.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a1_test=test.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()
print("Testing Sample2 % :",(test.User_id.count()/CheckinInfo.User_id.count())*100)

Testing Sample2 % : 9.976977920526307


In [0]:
# hitrate - one visit hide
rows = a1_train.User_id.astype(person_c).cat.codes
cols = a1_train.Item_id.astype(thing_c).cat.codes
sparse_checkin = csr_matrix((a1_train['count'], (rows, cols)),shape=(person_c.categories.size, thing_c.categories.size))

In [0]:
# hitrate - one item hide
import random
d1={} #all user-item
d2={} #hidden user-item

    
#generating d[user]=item(s)
for index,row in a1.iterrows():
    key1=row.User_id
    key2=row.Item_id
    if key1 not in dict_singleUser:
        if key1 not in d1.keys():
            d1[key1]=[key2]
        else:
            d1[key1].append(key2)

In [0]:
# hitrate - one item hide
import tqdm
list_toBeDeleted=[]
visited=[]

with tqdm.tqdm(total=a1.User_id.count()) as progress:
  for index,row in a1.iterrows():
      if row.User_id not in visited and row.User_id not in dict_singleUser:
          temp=random.choice(d1[row.User_id])
          d2[row.User_id]=temp
          d1[row.User_id].remove(temp)
          visited.append(row.User_id)
          list_toBeDeleted.append(index)
      progress.update(1)
print("Testing Sample1 % :",(len(user_lbl)/a1.Item_id.count())*100)
        


100%|██████████| 1992299/1992299 [23:46<00:00, 1396.72it/s]


Testing Sample1 % : 2.7932554300333434


In [0]:
# hitrate - one item hide
a1.drop(list_toBeDeleted, inplace=True)
rows = a1.User_id.astype(person_c).cat.codes
cols = a1.Item_id.astype(thing_c).cat.codes
sparse_checkin = csr_matrix((a1['count'], (rows, cols)),shape=(person_c.categories.size, thing_c.categories.size))

In [0]:
#als
import implicit
import numpy as np

model=implicit.als.AlternatingLeastSquares(factors=50,iterations=20)
model.approximate_similar_items = False

In [0]:
from implicit.nearest_neighbours import bm25_weight
alpha_val = 40
data_conf = (sparse_checkin * alpha_val).astype('double')
sparse_checkin_2 = bm25_weight(data_conf, K1=100, B=0.8)

In [0]:
import time
print("training model :als")
start = time.time()
model.fit(sparse_checkin_2.T.tocsr())
print("trained model in %0.2fs" %(time.time() - start))

training model :als


100%|██████████| 20.0/20 [00:01<00:00, 14.40it/s]

trained model in 1.87s


In [0]:
import tqdm
import codecs

#start = time.time()
output_filename = '/content/gdrive/My Drive/Dataset/recommends_implicit@50000.txt'
user_item = sparse_checkin_2.tocsr()

with tqdm.tqdm(total=len(user_lbl)) as progress:
    with open(output_filename, "w") as o:
        for u_id in range(len(user_lbl)):
            for i_id, score in model.recommend(u_id, user_item):
                o.write("%s\t%s\t%s\n" %(user_lbl[u_id], item_lbl[i_id], score))
            progress.update(1)           
print("generated recommendations in %0.2fs" %(time.time() - start))

100%|██████████| 50002/50002 [06:19<00:00, 131.62it/s]

generated recommendations in 381.82s


In [0]:
# hitrate - one item hide
dict_test={}
list_correct2=[]
n_users,n_items=sparse_checkin.shape
with open("/content/gdrive/My Drive/Dataset/recommends_implicit.txt") as g:
    for line in g:
        key1,key2,key3=line.split()
        key1=int(key1)
        if key1 not in dict_test.keys():
            dict_test[key1]=[key2]
        else:
            dict_test[key1].append(key2)
          
hit=0
print("\n----------------------------\n")
for key,value in dict_test.items():
  if key not in dict_singleUser:
    for item in value:
          if item in d2[key]:
              hit+=1
              list_correct2.append(key)
total=len(d2.keys())
print("\n",hit,"\n",hit/total) 
print(list_correct2)



----------------------------


 295 
 0.006310160427807487
[62868768, 1039014865, 1043618272, 1051317345, 1065880097, 1070862992, 1071740597, 1077232507, 1078809680, 1079575775, 1084006190, 1092865750, 1095100685, 1108512195, 1114213314, 1135646783, 1150137647, 1160027125, 1161209790, 1169298217, 1171845342, 1178381581, 1195568352, 1213241880, 1231359592, 1234576665, 1238247555, 1239711425, 1245242010, 1245376635, 1248773181, 1258046645, 1265148327, 1267512543, 1274979111, 1283017224, 1296046883, 1301242891, 1335204405, 1341857853, 1348495277, 1354795373, 1362366280, 1367956235, 1371460374, 1385519795, 1400781511, 1401516864, 1401850241, 1410368982, 1410549684, 1410559107, 1411696475, 1418082827, 1419987323, 1422651161, 1437589177, 1448846941, 1452428894, 1457095045, 1463641785, 1465285914, 1470832772, 1497535851, 1502846251, 1505911457, 1506354893, 1515944844, 1517806292, 1526448254, 1538119644, 1562698321, 1568314587, 1570143755, 1571895271, 1574409702, 1575286603, 1576081925, 15773

In [0]:
# hitrate - one visit hide
dict_test={}
list_correct2=[]
n_users,n_items=sparse_checkin.shape
with open("/content/gdrive/My Drive/Dataset/recommends_implicit@50000.txt",'r') as g:
  for line in g:
      key1,key2,key3=line.split()
      key1=int(key1)
      if key1 not in dict_test.keys():
          dict_test[key1]=[key2]
      else:
          dict_test[key1].append(key2)
          
d2={}          
for index,row in a1_test.iterrows():
    key1=row.User_id
    key2=row.Item_id
    if key1 not in d2.keys():
        d2[key1]=[key2]
    else:
        d2[key1].append(key2)
            

hit=0
hit_c=0
hit_r=0
hit_p=0

User_visit=a1.groupby(['User_id'])[["count"]].sum().reset_index()
d_user = {}
for index,row in User_visit.iterrows():
    d_user[row["User_id"]]=row["count"]

check_keys=list(d2.keys())
print("\n----------------------------\n")
for key,value in dict_test.items():
  if key in check_keys:
    for item in value:
          if item in d2[key]:
              hit+=1
              hit_c+=1
              list_correct2.append(key)
    hit_r+=(hit_c / d_user[key])
    hit_p+=(hit_c / 10)
    hit_c=0

  
total=len(d2.keys())
print("Total:",total,"\nHit:",hit,"\nHit Ratio:",hit/total,"\nRecall@10:",hit_r,"  ",hit_r/total,"\nPrecision@10:",hit_p,"  ",hit_p/total)
print(list_correct2)


----------------------------

Total: 30773 
Hit: 2890 
Hit Ratio: 0.0939134955967894 
Recall@10: 106.8308469103181    0.0034715772563714328 
Precision@10: 288.9999999999966    0.009391349559678828
[103984, 1002797063, 1002797063, 1004333025, 1009001235, 1011474317, 1012658495, 1013181182, 1013738042, 1013933125, 1013933125, 1017677027, 1018026622, 1018921635, 1026124163, 1027314712, 1030863794, 1030903703, 1032122190, 1032376922, 1033175317, 1034085112, 1034357363, 1034839023, 1035043242, 1035256354, 1036008415, 1036183655, 1036183655, 1037182015, 1039223673, 1042411705, 1043460474, 1043566247, 1044088437, 1045029787, 1045880445, 1046567003, 1046701711, 1046701711, 1047420117, 1047594024, 1048375381, 1048549455, 1049153021, 1049913277, 1050091053, 1051253283, 1051289890, 1052614217, 1052773033, 1053023500, 1053252387, 1053507862, 1054203734, 1054828894, 1055188054, 1056621315, 1056705060, 1057701355, 1061403407, 1061403407, 1062521357, 1063123334, 1064603551, 1064865674, 1065157600, 1

In [0]:
#TODO:  Evaluation of recommender
#https://stats.stackexchange.com/questions/226825/what-metric-should-i-use-for-assessing-implicit-matrix-factorization-recommender
#https://github.com/statisticianinstilettos/recmetrics
    
#TODO:  Metadata Embeddings for User and Item Cold-start Recommendations
#https://towardsdatascience.com/creating-a-hybrid-content-collaborative-movie-recommender-using-deep-learning-cc8b431618af
#https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243
#https://stackoverflow.com/questions/33757974/how-to-add-user-and-item-metadata-in-recommendation-engine-and-which-python-open
#https://github.com/jalajthanaki/Movie_recommendation_engine

#TODO:matchbox and SVD feature box
#https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/score-matchbox-recommender
#https://github.com/Azure-Readiness/hol-azure-machine-learning/blob/master/008-lab-recommendation-system.md
#https://github.com/beechung/Latent-Factor-Models
#https://github.com/Coder-Yu/RecQ


#recommender
#https://www.coursera.org/specializations/recommender-systems

#Matrix Factorization
#https://medium.com/@connectwithghosh/simple-matrix-factorization-example-on-the-movielens-dataset-using-pyspark-9b7e3f567536
#https://lazyprogrammer.me/tutorial-on-collaborative-filtering-and-matrix-factorization-in-python/
#http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

# TODO NOW:
# https://github.com/lyst/lightfm
    # https://github.com/jalajthanaki/Movie_recommendation_engine
#https://github.com/NicolasHug/Surprise
#https://github.com/DefuLian/recsys

#https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus